# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree.
* Test your function on the tuple `('yes',31,'good')`,

In [84]:
def decisionTreeClassifier(t):
    if(t[0] == 'yes'):
        return ('less' if t[1] <= 29.5 else 'more')
    else:
        return ('less' if t[2] == 'good' else 'more')
            
decisionTreeClassifier(('yes',31,'good'))

'more'

## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [85]:
def getFormattedDataFromFile(formatter, filename):
    return map(formatter, (r[:-1].split(',') for r in open(filename, 'r')))

testSetFormatter = lambda p: (p[0],int(p[1]),p[2])
testSet = getFormattedDataFromFile(testSetFormatter, 'health-test.txt')

testSet

[('yes', 21, 'poor'),
 ('no', 50, 'good'),
 ('no', 23, 'good'),
 ('yes', 45, 'poor'),
 ('yes', 51, 'good'),
 ('no', 60, 'good'),
 ('no', 15, 'poor'),
 ('no', 18, 'good')]

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and compute the percentage of them that are classified as "more risk".

In [86]:
percentageOfMoreRisk = lambda classifier, dataset: sum(s == 'more' for s in map(classifier, dataset))/float(len(dataset))
percentageOfMoreRisk(decisionTreeClassifier, testSet)

0.375

## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [87]:
# Use the function getFormattedDataFromFile defined in [56] in order to set the trainSet
trainSetFormatter = lambda p: ((p[0],int(p[1]),p[2]), p[3])
trainSet = getFormattedDataFromFile(trainSetFormatter, 'health-train.txt')
trainSet

[(('yes', 54, 'good'), 'less'),
 (('no', 55, 'good'), 'less'),
 (('no', 26, 'good'), 'less'),
 (('yes', 40, 'good'), 'more'),
 (('yes', 25, 'poor'), 'less'),
 (('no', 13, 'poor'), 'more'),
 (('no', 15, 'good'), 'less'),
 (('no', 50, 'poor'), 'more'),
 (('yes', 33, 'good'), 'more'),
 (('no', 35, 'good'), 'less'),
 (('no', 41, 'good'), 'less'),
 (('yes', 30, 'poor'), 'more'),
 (('no', 39, 'poor'), 'more'),
 (('no', 20, 'good'), 'less'),
 (('yes', 18, 'poor'), 'less'),
 (('yes', 55, 'good'), 'more')]

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a,b) = (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes',31,'good')`

In [88]:
def nearestNeighborClassifier(a):
    #distanceList stores all distances between trainSet and point a
    distance = lambda a,b: (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])
    distanceList = map(lambda b: distance(a,b[0]), trainSet)
    #Finding the minimal distance and returning the nearest point
    return trainSet[distanceList.index(min(distanceList))][1]

nearestNeighborClassifier(('yes',31,'good'))


'more'

* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and the fraction of the time it happens.

In [89]:
# The function compareDataPoints returns the set of points for wich all classifiers agree or disagree
def compareDataPoints(classifiers, dataset, decision):
    agree = (decision == 'agree')
    diff = zip(*[map(classifier, dataset) for classifier in classifiers])
    def compare(arr):
        for i in range(len(arr) - 1):
            if arr[i] != arr[i+1]:
                return not agree
        return agree
    return [testSet[k] for k,v in enumerate(diff) if compare(v)]

# Use compareDataPoints function in order to find points for which the two classifiers disagree
disagreePoints = compareDataPoints([decisionTreeClassifier, nearestNeighborClassifier], testSet, 'disagree')
(disagreePoints, len(disagreePoints) / float(len(testSet)))

([('yes', 51, 'good')], 0.125)

One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [90]:
class NearestMeanClassifier:
    
    # Training method that takes as input a dataset
    # and produces two internal vectors corresponding
    # to the mean of each class.
    
    trainedVectors = {
        'less' : (),
        'more' : ()
    }
    
    def distance(self, a, b):
        return (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2
    
    def adaptVector(self, point): 
        return (float(point[0]=='yes'), point[1], float(point[2]=='poor')) 
    
    def meanVector(self, points):
        return tuple(map(lambda x: float(x)/len(points), map(sum, zip(*points))))
    
    def getClassPoints(self, param, dataset):
        return [self.adaptVector(v[0]) for v in dataset if v[1] == param]
   
    def train(self, dataset):
        self.trainedVectors['more'] = self.meanVector(self.getClassPoints('more', dataset))
        self.trainedVectors['less'] = self.meanVector(self.getClassPoints('less', dataset))
        return self.trainedVectors
    
    # Prediction method that takes as input a new data
    # point and predicts it to belong to the class with
    # nearest mean.
    def predict(self,x):
        return 'more' if (self.distance(self.adaptVector(x), self.trainedVectors['more']) <= 
                self.distance(self.adaptVector(x), self.trainedVectors['less'])) else 'less'

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [91]:
# Create an object of NearestMeanClassifier and train it by trainSet
nearestMeanClassifier = NearestMeanClassifier()
nearestMeanClassifier.train(trainSet)

{'less': (0.3333333333333333, 32.111111111111114, 0.2222222222222222),
 'more': (0.5714285714285714, 37.142857142857146, 0.5714285714285714)}

* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [92]:
# Use function compareDataPoints defined in [60] in order to find points for which all three classifiers agree
compareDataPoints([decisionTreeClassifier, nearestNeighborClassifier, nearestMeanClassifier.predict], testSet, 'agree')

[('no', 50, 'good'),
 ('no', 23, 'good'),
 ('yes', 45, 'poor'),
 ('no', 60, 'good'),
 ('no', 15, 'poor'),
 ('no', 18, 'good')]